# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Importing establishment.json into uk_food database as the collection establishments:
    ```shell
    mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json
    ```

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [5]:
# assign the collection to a variable
establishments = db['establishments']

In [6]:
# review a document in the establishments collection
pprint(establishments.find_one())
#printing first result to look into possible fields

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('647663a07ebe78a8a0dff5e7'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
# Provided new object for the collection
update = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}


In [9]:
#inserting the new object to the collection
establishments.insert_one(update)

In [10]:
# Check that the new restaurant was inserted
establishments.find_one({"BusinessName":"Penang Flavours"})

{'_id': ObjectId('6476640805d8e6919f3cf62a'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
query = {'BusinessType':'Restaurant/Cafe/Canteen'}
fields = {"BusinessTypeID": 1,"BusinessType": 1}
#Finding the BusinessTypeID for "Restaurant/Cafe/Canteen" 
#and returning only the BusinessTypeID and BusinessType fields.
results = establishments.find(query, projection=fields)
pprint(results[0])

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('647663a07ebe78a8a0dff5e7')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
business_id = results[0]['BusinessTypeID']
establishments.update_one({"BusinessName":"Penang Flavours"},{'$set':{'BusinessTypeID':business_id}})

In [13]:
# Confirm that the new restaurant was updated
fields = {"BusinessTypeID": 1,"BusinessName": 1}
pprint(establishments.find_one({"BusinessName":"Penang Flavours"},projection=fields))

{'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1,
 '_id': ObjectId('6476640805d8e6919f3cf62a')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
#count current documents
print(establishments.count_documents({}))
#find how many focuments contain the Dover Local Authority
#'LocalAuthorityName': 'Dover'
query = {'LocalAuthorityName': 'Dover'}
print(establishments.count_documents(query))

39780
994


In [15]:
# Remove query matches
establishments.delete_many(query)
#count remaining documents
print(establishments.count_documents({}))
#successfully removed dover

38786


In [16]:
# Check if any remaining documents include Dover
print(establishments.count_documents(query))

0


In [17]:
# Check that other documents remain with 'find_one'
print(establishments.count_documents({}))

38786


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({},[{"$set": 
                                {
                                   "geocode.longitude": { "$toDecimal": "$geocode.longitude" },
                                    "geocode.latitude": { "$toDecimal": "$geocode.latitude" }
                                 }
                               }])

Use `update_many` to convert `RatingValue` to integer numbers.

In [19]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [20]:
#set rating value to integer
establishments.update_many({},[{"$set": 
                                {
                                   "RatingValue": { "$toInt": "$RatingValue" }
                                 }
                               }])

In [29]:
# Check that the coordinates and rating value are now numbers
print(type(establishments.find_one({},{'RatingValue':1})['RatingValue']))
print(type(establishments.find_one({},{'geocode.latitude':1})['geocode']['latitude']))
print(type(establishments.find_one({},{'geocode.longitude':1})['geocode']['longitude']))

<class 'int'>
<class 'bson.decimal128.Decimal128'>
<class 'bson.decimal128.Decimal128'>
